# Retail Customer Segmentation & Sales Analysis

**Note** - This notebok uses a **representative sample** of the original retail transactions dataset. The original dataset contains over 1000000 rows, but this sample contains **5000 rows** for memory-friendly processing and Github sharing.

## Project Overview
This project analyzes retail transaction data to understand customer purchasing behaviour, identify high-level customer segments, generate actionable insights, and evaluate the impact of several factors like discounts, promotions and more on sales performance.

## Business Problem 
Retail businesses need to understand how customers behave across different store formats, promotions, and seasons in order to improve targeting, optimize discount strategies, and increase overall revenue. They also want to identify high-value segments, and optimize promotions to increase revenue.

## Project Objectives
- Clean and preprocess raw transaction data
- Normalize transaction-level data into analysis-ready tables
- Create customer-level metrics and segements
- Analyze how customer behaviour varies by several factors
- Build an interactive dashboard
- Generate actionable insights for business decision-making

## Dataset Description
The dataset contains retail transaction-level data with 12 columns describing customer, product, and transaction attributes.

### Dataset Columns

| Column Name | Description |
|-------------|-------------|
| transaction_id | Unique identifier for each transaction |
| date | Date and time of purchase |
| customer_name | Name of the customer |
| product | List of products purchased in production |
| total_items | Total number of items in transaction |
| total_cost | Total monetary value of transaction |
| payment_method | Mode of payment (cash, card, etc.) |
| city | City where the transaction occured |
| store_type | Type of store (warehouse, department, etc.) |
| discount_applied | Indicates whether discount is applied (T/F) |
| customer_category | Type of customer (Professional, homemaker, etc.) |
| season | Season during which purchase occured |
| promotion | Promotion applied (None, BOGO, etc.) |

## Key Business Questions
1. How can customers be segmented based on their spending behaviour?
2. How do discounts and promotions influence total transaction value?
3. Which store types and cities generate the highest revenue?
4. How does customer purchasing behaviour vary across seasons?
5. Which customer type contribute most to overall sales?

## Notebook Scope 
This notebook focuses on data loading, cleaning, validation and restructuring.

In [2]:
# Importing required libraries
import pandas as pd
import numpy as np

## Data Loading

In [3]:
file_path = r"C:\Users\abc\Documents\projects\Retail_Transactions_Dataset.csv"
df_raw = pd.read_csv(file_path)
df_raw.head()

,Transaction_ID,Date,Customer_Name,Product,Total_Items,Total_Cost,Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion
0,1000000000,2022-01-21 06:27:29,Stacey Price,"['Ketchup', 'Shaving Cream', 'Light Bulbs']",3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,NaN
1,1000000001,2023-03-01 13:01:21,Michelle Carlson,"['Ice Cream', 'Milk', 'Olive Oil', 'Bread', 'P...",2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One)
2,1000000002,2024-03-21 15:37:04,Lisa Graves,['Spinach'],6,41.49,Credit Card,Houston,Department Store,True,Professional,Winter,NaN
3,1000000003,2020-10-31 09:59:47,Mrs. Patricia May,"['Tissues', 'Mustard']",1,39.34,Mobile Payment,Chicago,Pharmacy,True,Homemaker,Spring,NaN
4,1000000004,2020-12-10 00:59:59,Susan Mitchell,['Dish Soap'],10,16.42,Debit Card,Houston,Specialty Store,False,Young Adult,Winter,Discount on Selected Items


In [4]:
df_sample = df_raw.iloc[:5000].copy()
df_sample.to_csv(r"C:\Users\abc\Documents\projects\Retail-Customer_Segmentation-and-Sales-Analysis\data\sample\Retail_Transaction_Sample.csv", index=False)

## Initial Data inspection

In [5]:
df_sample.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Transaction_ID     5000 non-null   int64  
 1   Date               5000 non-null   object 
 2   Customer_Name      5000 non-null   object 
 3   Product            5000 non-null   object 
 4   Total_Items        5000 non-null   int64  
 5   Total_Cost         5000 non-null   float64
 6   Payment_Method     5000 non-null   object 
 7   City               5000 non-null   object 
 8   Store_Type         5000 non-null   object 
 9   Discount_Applied   5000 non-null   bool   
 10  Customer_Category  5000 non-null   object 
 11  Season             5000 non-null   object 
 12  Promotion          3287 non-null   object 
dtypes: bool(1), float64(1), int64(2), object(9)
memory usage: 473.8+ KB


#### **Key Insights**
1. The dataset contains 13 columns and 5000 rows.
2. Most columns are stored as object types.
3. The date column will be converted to datetime format for time-based analysis.
4. The promotion column contains fewer non-null values indicating missing entries.

In [6]:
df_sample['Promotion'].isnull().sum()

np.int64(1713)

In [7]:
df_sample.describe()

,Transaction_ID,Total_Items,Total_Cost
count,5.000000e+03,5000.000000,5000.000000
mean,1.000002e+09,5.545000,52.565994
std,1.443520e+03,2.858043,27.483945
min,1.000000e+09,1.000000,5.000000
25%,1.000001e+09,3.000000,28.880000
50%,1.000002e+09,6.000000,52.490000
75%,1.000004e+09,8.000000,76.162500
max,1.000005e+09,10.000000,99.980000


#### **Key Insights**
1. The maximum number of items purchased is 10 and the minimum is 1.
2. The highest payment made by a customer is 99.98, whereas the lowest is 5.00.

## Data Cleaning and Transformation

#### Dealing with missing values

There are missing values present in Promotion column, which likely indicates that no promotions were available. This will be handled accordingly

In [8]:
df_sample['Promotion'] = df_sample['Promotion'].fillna('No Promotion')

In [9]:
df_sample['Promotion'].head(5)

0                  No Promotion
1        BOGO (Buy One Get One)
2                  No Promotion
3                  No Promotion
4    Discount on Selected Items
Name: Promotion, dtype: object

In [10]:
df_sample['Promotion'].isna().sum()

np.int64(0)

#### Date and Time processing

The transaction timestamp was originally stored as a single object column containing both date and time. This column was converted to datetime format and split into seperate date and time columns.

In [11]:
df_sample['Date'] = pd.to_datetime(df_sample['Date'], format = '%Y-%m-%d %H:%M:%S', errors='coerce')

In [12]:
df_sample["Date"].head()

0   2022-01-21 06:27:29
1   2023-03-01 13:01:21
2   2024-03-21 15:37:04
3   2020-10-31 09:59:47
4   2020-12-10 00:59:59
Name: Date, dtype: datetime64[ns]

In [13]:
df_sample['Transaction_date'] = df_sample['Date'].dt.date
df_sample['Transaction_time'] = df_sample['Date'].dt.time

In [14]:
df_sample = df_sample.drop(columns=['Date'])

In [15]:
df_sample[['Transaction_ID', 'Transaction_date', 'Transaction_time']].head(3)

,Transaction_ID,Transaction_date,Transaction_time
0,1000000000,2022-01-21,06:27:29
1,1000000001,2023-03-01,13:01:21
2,1000000002,2024-03-21,15:37:04


#### Customer Identification

A customer identifier was created to uniquely identify customers based on their names.

In [16]:
df_sample["Customer_ID"] = pd.factorize(df_sample['Customer_Name'])[0]+1
cols = ['Transaction_ID','Customer_ID','Customer_Name']+[col for col in df_sample.columns if col not in ['Transaction_ID','Customer_ID','Customer_Name']]
df_sample = df_sample[cols]

In [17]:
df_sample[['Transaction_ID','Customer_ID','Customer_Name']].head(5)

,Transaction_ID,Customer_ID,Customer_Name
0,1000000000,1,Stacey Price
1,1000000001,2,Michelle Carlson
2,1000000002,3,Lisa Graves
3,1000000003,4,Mrs. Patricia May
4,1000000004,5,Susan Mitchell


#### Produst list normalization (Exploding Product column)

The product column contained multiple items in a list format. It will be exploded so that each product appears in a seperate row while retaining the same Transaction ID.

In [18]:
df_sample["Product"].head(3)

0          ['Ketchup', 'Shaving Cream', 'Light Bulbs']
1    ['Ice Cream', 'Milk', 'Olive Oil', 'Bread', 'P...
2                                          ['Spinach']
Name: Product, dtype: object

In [19]:
import ast

In [20]:
df_sample['Product'] = df_sample['Product'].apply(ast.literal_eval)

In [21]:
df_exploded = df_sample.explode('Product')

In [22]:
df_exploded = df_exploded.reset_index(drop=True)

In [23]:
df_exploded.head(5)

,Transaction_ID,Customer_ID,Customer_Name,Product,Total_Items,Total_Cost,Payment_Method,City,Store_Type,Discount_Applied,Customer_Category,Season,Promotion,Transaction_date,Transaction_time
0,1000000000,1,Stacey Price,Ketchup,3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,No Promotion,2022-01-21,06:27:29
1,1000000000,1,Stacey Price,Shaving Cream,3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,No Promotion,2022-01-21,06:27:29
2,1000000000,1,Stacey Price,Light Bulbs,3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Homemaker,Winter,No Promotion,2022-01-21,06:27:29
3,1000000001,2,Michelle Carlson,Ice Cream,2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One),2023-03-01,13:01:21
4,1000000001,2,Michelle Carlson,Milk,2,25.93,Cash,San Francisco,Specialty Store,True,Professional,Fall,BOGO (Buy One Get One),2023-03-01,13:01:21


#### Cleaned dataset ready for analysis

The dataset has been cleaned and prepared for analysis. The following steps create analytical dataframes at different levels to support business insights.

## Analytical Data Preparation

The transactions and products in the dataset exist at different levels of granularity. Exploding product changes the row meaning, so they are seperated into different dataframes to prevent duplicated totals and enables correct aggregtion

#### Transaction-level Dataframe

In [24]:
df_transactions = (df_sample.drop_duplicates(subset=['Transaction_ID'])
                   [['Transaction_ID', 'Customer_ID', 'Transaction_date', 'Transaction_time',
       'Total_Items', 'Total_Cost', 'Payment_Method', 'City', 'Store_Type',
       'Discount_Applied', 'Season', 'Promotion'
       ]])

In [25]:
df_transactions.head()

,Transaction_ID,Customer_ID,Transaction_date,Transaction_time,Total_Items,Total_Cost,Payment_Method,City,Store_Type,Discount_Applied,Season,Promotion
0,1000000000,1,2022-01-21,06:27:29,3,71.65,Mobile Payment,Los Angeles,Warehouse Club,True,Winter,No Promotion
1,1000000001,2,2023-03-01,13:01:21,2,25.93,Cash,San Francisco,Specialty Store,True,Fall,BOGO (Buy One Get One)
2,1000000002,3,2024-03-21,15:37:04,6,41.49,Credit Card,Houston,Department Store,True,Winter,No Promotion
3,1000000003,4,2020-10-31,09:59:47,1,39.34,Mobile Payment,Chicago,Pharmacy,True,Spring,No Promotion
4,1000000004,5,2020-12-10,00:59:59,10,16.42,Debit Card,Houston,Specialty Store,False,Winter,Discount on Selected Items


#### Customer-level Dataframe

In [26]:
customers_base = df_sample.groupby('Customer_ID').agg(
    Customer_Name=('Customer_Name','first'),
    Customer_Category=('Customer_Category','first')).reset_index()

In [27]:
customer_metrics = (df_transactions.groupby('Customer_ID',as_index=False).agg(
    Total_Spend = ('Total_Cost','sum'),
    Avg_Basket_Value = ('Total_Cost','mean'),
    Total_Transactions = ('Transaction_ID', 'nunique')
)).reset_index()

In [28]:
df_customers = customers_base.merge(customer_metrics, on='Customer_ID', how='left')

In [29]:
df_customers.head()

,Customer_ID,Customer_Name,Customer_Category,index,Total_Spend,Avg_Basket_Value,Total_Transactions
0,1,Stacey Price,Homemaker,0,71.65,71.65,1
1,2,Michelle Carlson,Professional,1,25.93,25.93,1
2,3,Lisa Graves,Professional,2,41.49,41.49,1
3,4,Mrs. Patricia May,Homemaker,3,39.34,39.34,1
4,5,Susan Mitchell,Young Adult,4,16.42,16.42,1


#### Product-level Dataframe

In [30]:
df_products = df_exploded[['Transaction_ID','Customer_ID','Product']].reset_index(drop=True)

In [31]:
df_products.head()

,Transaction_ID,Customer_ID,Product
0,1000000000,1,Ketchup
1,1000000000,1,Shaving Cream
2,1000000000,1,Light Bulbs
3,1000000001,2,Ice Cream
4,1000000001,2,Milk


## Summary

This notebook performs all necessary data cleaning and transformation to prepare the retails sales dataset for analysis.

**Key Steps Completed**
1. **Data Cleaning and Transformation**
    - Handled missing values and formatted columns
    - Split Transaction date and time
    - Created Customer ID
    - Exploded products list into individual rows

2. **Analytical Data Preparation**
    - Created 'df_transactions' (1 row per transaction)
    - Created 'df_customers' (1 row per customer)
    - Created 'df_products' (1 row per product)

**Notes**
- Metrics in 'df_customers' are based on the sample dataset.
- Data is ready for analysis in the next notebook

In [32]:
df_transactions.to_csv(r"C:\Users\abc\Documents\projects\Retail-Customer_Segmentation-and-Sales-Analysis\data\processed\transactions.csv", index=False)
df_customers.to_csv(r"C:\Users\abc\Documents\projects\Retail-Customer_Segmentation-and-Sales-Analysis\data\processed\customers.csv", index=False)
df_products.to_csv(r"C:\Users\abc\Documents\projects\Retail-Customer_Segmentation-and-Sales-Analysis\data\processed\products.csv", index=False)